SQL
---

Let's drop everything into a table, and from there we can pick out the unique station identifiers (unit), produce the difference columns, do our cleaning, and create a new table (in a separate database) that has our cleaned data. Yes, this should have been done from the get-go, but better late than never...

In [3]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

database = 'raw_stations'
user = 'mikemoran'

In [4]:
engine = create_engine(f'postgres://{user}@localhost/{database}')
if not database_exists(engine.url):
    create_database(engine.url)

In [10]:
import os

files = [f'raw_data/{f}' for f in sorted(os.listdir('raw_data/'))]
split = 'raw_data/turnstile_141018.txt'
modern_files = files[files.index(split):]
modern_files[:5]

['raw_data/turnstile_141018.txt',
 'raw_data/turnstile_141025.txt',
 'raw_data/turnstile_141101.txt',
 'raw_data/turnstile_141108.txt',
 'raw_data/turnstile_141115.txt']

Modern Style
--------------

In [11]:
def load_turnstile(filename):
    df = pd.read_csv(filename, header=0,
                     parse_dates=[['DATE', 'TIME']], infer_datetime_format=True,)
    df.columns = df.columns.str.strip().str.lower().str.replace('/', '_')
    return df

In [12]:
for filename in modern_files:
    df = load_turnstile(filename)
    df.to_sql('raw', engine, if_exists='append')

In [16]:
conn = None
conn = psycopg2.connect(database=database, user=user)

query = '''
    select * from raw
    where unit = 'R072';'''
r072 = pd.read_sql(query, conn)
r072.head()

,index,date_time,c_a,unit,scp,station,linename,division,desc,entries,exits
0,182004,2016-05-07 00:00:00,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,REGULAR,8267,35122
1,182005,2016-05-07 04:00:00,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,REGULAR,8267,35136
2,182006,2016-05-07 08:00:00,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,REGULAR,8267,35140
3,182007,2016-05-07 12:00:00,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,REGULAR,8270,35161
4,182008,2016-05-07 16:00:00,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,REGULAR,8274,35186


Old Style
-----------

In [18]:
old_files = files[:files.index(split)]
old_files = list(filter(lambda x: 'turnstile' in x, old_files))
old_files[:5]

['raw_data/turnstile_100505.txt',
 'raw_data/turnstile_100508.txt',
 'raw_data/turnstile_100515.txt',
 'raw_data/turnstile_100522.txt',
 'raw_data/turnstile_100605.txt']

In [21]:
test = pd.read_csv(old_files[0], header=None)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,A002,R051,02-00-00,04-17-10,00:00:00,REGULAR,2704717,928793,04-17-10,04:00:00,...,04-18-10,00:00:00,REGULAR,2705426.0,928987.0,04-18-10,04:00:00,REGULAR,2705426.0,928987.0
1,A002,R051,02-00-00,04-18-10,08:00:00,REGULAR,2705436,929002,04-18-10,12:00:00,...,04-19-10,08:00:00,REGULAR,2705937.0,929229.0,04-19-10,12:00:00,REGULAR,2706063.0,929533.0
2,A002,R051,02-00-00,04-19-10,16:00:00,REGULAR,2706335,929588,04-19-10,20:00:00,...,04-20-10,08:00:00,DOOR,2707246.0,929705.0,04-20-10,12:00:00,OPEN,2707385.0,930006.0
3,A002,R051,02-00-00,04-20-10,16:00:00,REGULAR,2707710,930067,04-20-10,20:00:00,...,04-21-10,16:00:00,REGULAR,2709171.0,930618.0,04-21-10,18:40:52,DOOR,2709868.0,930669.0
4,A002,R051,02-00-00,04-21-10,20:00:00,REGULAR,2710045,930698,04-22-10,00:00:00,...,04-22-10,20:00:00,REGULAR,2711562.0,931222.0,04-23-10,00:00:00,REGULAR,2711702.0,931240.0


In [25]:
# pd.read_csv(old_files[0], header=None, usecols=[0, 1, 2, 3, 4, 5, 6, 7])
# pd.read_csv(old_files[0], header=None, usecols=[0, 1, 2, 8, 9, 10, 11, 12])
# pd.read_csv(old_files[0], header=None, usecols=[0, 1, 2, 13, 14, 15, 16, 17])

In [36]:
import itertools

start = [0, 1, 2]
iterable = range(3, 43)
n = 5
fillvalue = 0
remainders = list(itertools.zip_longest(*[iter(iterable)]*n, fillvalue=fillvalue))
column_vals = [[*start, *row] for row in remainders]
list(column_vals)

[[0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2, 8, 9, 10, 11, 12],
 [0, 1, 2, 13, 14, 15, 16, 17],
 [0, 1, 2, 18, 19, 20, 21, 22],
 [0, 1, 2, 23, 24, 25, 26, 27],
 [0, 1, 2, 28, 29, 30, 31, 32],
 [0, 1, 2, 33, 34, 35, 36, 37],
 [0, 1, 2, 38, 39, 40, 41, 42]]

In [67]:
# for a single file, need eight calls to pd.read_csv
column_names = ['c_a', 'unit', 'scp', 'date', 'time', 'desc', 'entries', 'exits']
frames = [pd.read_csv(old_files[0], header=None, usecols=usecols, names=column_names)
          for usecols in column_vals]
full_frame = pd.concat(frames, axis=0)
full_frame['date_time'] = pd.to_datetime(
    (full_frame.date + full_frame.time).str.strip(),
    format='%m-%d-%y%H:%M:%S')
full_frame.dropna(inplace=True)
full_frame.reset_index(drop=True, inplace=True)
full_frame.drop(['date', 'time'], inplace=True, axis=1)
full_frame.tail()

,c_a,unit,scp,desc,entries,exits,date_time
237010,S102,R165,00-05-01,REGULAR,75.0,0.0,2010-04-18 02:00:00
237011,S102,R165,00-05-01,REGULAR,76.0,0.0,2010-04-19 10:00:00
237012,S102,R165,00-05-01,REGULAR,76.0,0.0,2010-04-20 18:00:00
237013,S102,R165,00-05-01,REGULAR,76.0,0.0,2010-04-22 02:00:00
237014,S102,R165,00-05-01,REGULAR,76.0,0.0,2010-04-23 07:07:53


In [92]:
def load_turnstile_old(filename, **kwargs):
    frames = [pd.read_csv(filename, header=None, usecols=usecols,
                          names=column_names, **kwargs)
              for usecols in column_vals]
    full_frame = pd.concat(frames, axis=0)
    full_frame.dropna(inplace=True)
    full_frame['date_time'] = pd.to_datetime(
        full_frame.date + full_frame.time,
        format='%m-%d-%y%H:%M:%S')
    full_frame.drop(['date', 'time'], axis=1, inplace=True)
    full_frame.reset_index(drop=True, inplace=True)
    return full_frame

In [69]:
for filename in old_files:
    print(filename)
    df = load_turnstile_old(filename)
    df.to_sql('raw', engine, if_exists='append')

raw_data/turnstile_100505.txt
raw_data/turnstile_100508.txt
raw_data/turnstile_100515.txt
raw_data/turnstile_100522.txt
raw_data/turnstile_100605.txt
raw_data/turnstile_100612.txt
raw_data/turnstile_100619.txt
raw_data/turnstile_100626.txt
raw_data/turnstile_100703.txt
raw_data/turnstile_100710.txt
raw_data/turnstile_100717.txt
raw_data/turnstile_100724.txt
raw_data/turnstile_100731.txt
raw_data/turnstile_100807.txt
raw_data/turnstile_100814.txt
raw_data/turnstile_100821.txt
raw_data/turnstile_100828.txt
raw_data/turnstile_100904.txt
raw_data/turnstile_100911.txt
raw_data/turnstile_100918.txt
raw_data/turnstile_100925.txt
raw_data/turnstile_101002.txt
raw_data/turnstile_101009.txt
raw_data/turnstile_101016.txt
raw_data/turnstile_101023.txt
raw_data/turnstile_101030.txt
raw_data/turnstile_101106.txt
raw_data/turnstile_101113.txt
raw_data/turnstile_101120.txt
raw_data/turnstile_101127.txt
raw_data/turnstile_101204.txt
raw_data/turnstile_101211.txt
raw_data/turnstile_101218.txt
raw_data/t

/opt/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


DataError: (psycopg2.DataError) invalid input syntax for integer: "-"
LINE 1: ...601', 'R319', '00-03-00', 'DOOR OPEN', 2814000.0, '-', '2012...
                                                             ^
 [SQL: 'INSERT INTO raw (index, c_a, unit, scp, "desc", entries, exits, date_time) VALUES (%(index)s, %(c_a)s, %(unit)s, %(scp)s, %(desc)s, %(entries)s, %(exits)s, %(date_time)s)'] [parameters: ({'index': 0, 'c_a': 'A002', 'unit': 'R051', 'scp': '02-00-00', 'desc': 'REGULAR', 'entries': 3618144.0, 'exits': 1249112, 'date_time': datetime.datetime(2012, 4, 28, 0, 0)}, {'index': 1, 'c_a': 'A002', 'unit': 'R051', 'scp': '02-00-00', 'desc': 'REGULAR', 'entries': 3619169.0, 'exits': 1249424, 'date_time': datetime.datetime(2012, 4, 29, 4, 0)}, {'index': 2, 'c_a': 'A002', 'unit': 'R051', 'scp': '02-00-00', 'desc': 'REGULAR', 'entries': 3619991.0, 'exits': 1249972, 'date_time': datetime.datetime(2012, 4, 30, 12, 0)}, {'index': 3, 'c_a': 'A002', 'unit': 'R051', 'scp': '02-00-00', 'desc': 'RE

In [70]:
split = old_files[old_files.index('raw_data/turnstile_120505.txt'):]
split[:5]

['raw_data/turnstile_120505.txt',
 'raw_data/turnstile_120512.txt',
 'raw_data/turnstile_120519.txt',
 'raw_data/turnstile_120526.txt',
 'raw_data/turnstile_120602.txt']

In [78]:
test = load_turnstile_old(split[0])
# test.exits = test.exits.str.replace('-', '')
test.exits = pd.to_numeric(test.exits, errors='coerce')
test.head()

/opt/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,c_a,unit,scp,desc,entries,exits,date_time
0,A002,R051,02-00-00,REGULAR,3618144.0,1249112.0,2012-04-28 00:00:00
1,A002,R051,02-00-00,REGULAR,3619169.0,1249424.0,2012-04-29 04:00:00
2,A002,R051,02-00-00,REGULAR,3619991.0,1249972.0,2012-04-30 12:00:00
3,A002,R051,02-00-00,REGULAR,3622745.0,1250665.0,2012-05-01 20:00:00
4,A002,R051,02-00-00,REGULAR,3623480.0,1251176.0,2012-05-02 16:00:00


In [79]:
for filename in split:
    print(filename)
    df = load_turnstile_old(filename)
    df.exits = pd.to_numeric(df.exits, errors='coerce')
    df.to_sql('raw', engine, if_exists='append')

raw_data/turnstile_120505.txt


/opt/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


raw_data/turnstile_120512.txt
raw_data/turnstile_120519.txt
raw_data/turnstile_120526.txt
raw_data/turnstile_120602.txt
raw_data/turnstile_120609.txt
raw_data/turnstile_120616.txt
raw_data/turnstile_120623.txt
raw_data/turnstile_120630.txt
raw_data/turnstile_120707.txt
raw_data/turnstile_120714.txt


ValueError: time data '2NUMBER' does not match format '%m-%d-%y%H:%M:%S' (match)

In [82]:
test = pd.read_csv('raw_data/turnstile_120714.txt', skiprows=10, header=None)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,A002,R051,02-00-00,07-11-12,16:00:00,REGULAR,3709772,1282349,07-11-12,20:00:00,...,07-12-12,16:00:00,REGULAR,3711323.0,1282916.0,07-12-12,20:00:00,REGULAR,3712234.0,1282975.0
1,A002,R051,02-00-00,07-13-12,00:00:00,REGULAR,3712452,1282996,07-13-12,04:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A002,R051,02-00-01,07-07-12,00:00:00,REGULAR,3580147,769278,07-07-12,04:00:00,...,07-08-12,00:00:00,REGULAR,3581180.0,769400.0,07-08-12,04:00:00,REGULAR,3581222.0,769403.0
3,A002,R051,02-00-01,07-08-12,08:00:00,REGULAR,3581248,769419,07-08-12,12:00:00,...,07-08-12,16:00:00,REGULAR,3581492.0,769495.0,07-08-12,20:00:00,REGULAR,3581683.0,769513.0
4,A002,R051,02-00-01,07-09-12,00:00:00,REGULAR,3581747,769533,07-09-12,04:00:00,...,07-10-12,00:00:00,REGULAR,3583049.0,769797.0,07-10-12,04:00:00,REGULAR,3583069.0,769799.0


In [86]:
last_file = 'raw_data/turnstile_120714.txt'
split = old_files[old_files.index(last_file):]

filename = split.pop(0)
print(filename)
df = load_turnstile_old(filename, skiprows=10)
df.exits = pd.to_numeric(df.exits, errors='coerce')
df.to_sql('raw', engine, if_exists='append')

for filename in split:
    print(filename)
    df = load_turnstile_old(filename)
    df.exits = pd.to_numeric(df.exits, errors='coerce')
    df.to_sql('raw', engine, if_exists='append')

raw_data/turnstile_120714.txt
raw_data/turnstile_120721.txt
raw_data/turnstile_120728.txt
raw_data/turnstile_120804.txt
raw_data/turnstile_120811.txt
raw_data/turnstile_120818.txt
raw_data/turnstile_120825.txt


ValueError: time data 'FF-05-4008-22-12' does not match format '%m-%d-%y%H:%M:%S' (match)

In [91]:
pd.read_csv('raw_data/turnstile_120825.txt', engine='c').tail()

,A002,R051,02-00-00,08-18-12,00:00:00,REGULAR,003759779,001297676,08-18-12.1,04:00:00,...,08-19-12,00:00:00.1,REGULAR.6,003760494,001297958,08-19-12.1,04:00:00.1,REGULAR.7,003760525,001297962
28905,TRAM2,R469,00-05-01,08-20-12,17:00:00,REGULAR,5554,86,08-20-12,21:00:00,...,08-21-12,17:00:00,REGULAR,5554.0,86.0,08-21-12,21:00:00,REGULAR,5554.0,86.0
28906,TRAM2,R469,00-05-01,08-22-12,01:00:00,REGULAR,5554,86,08-22-12,05:00:00,...,08-23-12,01:00:00,REGULAR,5554.0,86.0,08-23-12,05:00:00,REGULAR,5554.0,86.0
28907,TRAM2,R469,00-05-01,08-23-12,09:00:00,REGULAR,5554,86,08-23-12,13:00:00,...,08-24-12,09:00:00,REGULAR,5554.0,86.0,08-24-12,13:00:00,REGULAR,5554.0,86.0
28908,TRAM2,R469,00-05-01,08-24-12,17:00:00,REGULAR,5554,86,08-24-12,21:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28909,USE,NaN,R328,FF-05-40,08-22-12,16:02:48,101,-16711708,-016763243 ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
last_file = 'raw_data/turnstile_120825.txt'
split = old_files[old_files.index(last_file):]

filename = split.pop(0)
print(filename)
df = load_turnstile_old(filename, skiprows=10)
df.exits = pd.to_numeric(df.exits, errors='coerce')
df.to_sql('raw', engine, if_exists='append')

for filename in split:
    print(filename)
    df = load_turnstile_old(filename)
    df.exits = pd.to_numeric(df.exits, errors='coerce')
    df.to_sql('raw', engine, if_exists='append')

raw_data/turnstile_120825.txt
raw_data/turnstile_120901.txt
raw_data/turnstile_120908.txt
raw_data/turnstile_120915.txt
raw_data/turnstile_120922.txt
raw_data/turnstile_120929.txt
raw_data/turnstile_121006.txt
raw_data/turnstile_121013.txt
raw_data/turnstile_121020.txt
raw_data/turnstile_121027.txt
raw_data/turnstile_121103.txt
raw_data/turnstile_121110.txt
raw_data/turnstile_121117.txt
raw_data/turnstile_121124.txt
raw_data/turnstile_121201.txt
raw_data/turnstile_121208.txt
raw_data/turnstile_121215.txt
raw_data/turnstile_121222.txt
raw_data/turnstile_121229.txt
raw_data/turnstile_130105.txt
raw_data/turnstile_130112.txt
raw_data/turnstile_130119.txt
raw_data/turnstile_130126.txt
raw_data/turnstile_130202.txt
raw_data/turnstile_130209.txt
raw_data/turnstile_130216.txt
raw_data/turnstile_130223.txt
raw_data/turnstile_130302.txt
raw_data/turnstile_130309.txt
raw_data/turnstile_130316.txt
raw_data/turnstile_130323.txt
raw_data/turnstile_130330.txt
raw_data/turnstile_130406.txt
raw_data/t

All files loaded into raw database. Now, we can pull and process to create clean tables in a new database. We'll have a table mapping station names to unit numbers and control areas, and a table for each station with times and riderships.